In [126]:
# 2026/2/6
# zhangzhong


In [127]:
# define the env

class Environment:
    def __init__(self, goal: int = 4):
        self.goal: int = goal
        self.state: int = 0
        self.actions = ["LEFT", "RIGHT"]


    def step(self, action: str) -> tuple[float, int]:
        if action == "LEFT":
            self.state = max(0, self.state-1)
        elif action == "RIGHT":
            self.state = min(self.goal, self.state+1)
        return (10 if self.is_done() else -1, self.state)
    
    def is_done(self) -> bool:
        return self.state == self.goal
    
    def reset(self):
        self.state = 0

In [128]:
test_env = Environment()
test_env.reset()
print(test_env.state)
test_env.step("LEFT")
assert test_env.state == 0

test_env.step("RIGHT")
assert test_env.state == 1

test_env.step("RIGHT")
test_env.step("RIGHT")
test_env.step("RIGHT")
test_env.step("RIGHT")
test_env.step("RIGHT")
test_env.step("RIGHT")
assert test_env.state == 4

0


In [129]:
## agent

import random

class Agent:
    def __init__(self, env: Environment, epsilon: float = 0.1):
        self.Q: dict[int, dict[str, float]] = {}
        self.env = env
        self.epsilon = epsilon

        ## init Q
        self.goal = env.goal
        self.actions = env.actions

        ##!!! BUG, should be range(env.goal + 1) instead of range(env.goal)
        # cause Q[next_state] may access env.goal+1
        for i in range(env.goal + 1):
            self.Q[i] = {action: 0 for action in env.actions}
        
    def sample_action(self, state: int) -> str:
        if random.random() < self.epsilon:
            return random.choice(self.env.actions)
        else:
            return max(self.Q[state], key=self.Q[state].get)
        

In [ ]:
## let's start training

goal = 4
env = Environment(goal=goal)
agent = Agent(env=env)
learning_rate = 0.1
discount_factor = 0.9
reward = 0

max_epochs = 100

##BUG this should in each episode
# curr_state: int = 0
# curr_action: str = agent.sample_action(state=curr_state)

for epoch in range(max_epochs):
    env.reset()
    ##BUG fix, should at here
    curr_state: int = 0
    curr_action: str = agent.sample_action(state=curr_state)

    for t in range(100):
        # curr_state = env.state
        # # sample action

        # curr_action = agent.sample_action(state=curr_state)
        if env.is_done():
            ##BUG chatgpt told me to update the end state
            # but I found, to update it or not, does not affect the policy
            # terminal update
            TD = reward - agent.Q[curr_state][curr_action]
            agent.Q[curr_state][curr_action] += learning_rate * TD
            break

        # take this action in env
        reward, next_state = env.step(action=curr_action)
        # we also sample next action
        next_action = agent.sample_action(state=next_state)

        # go to key point of SARSA
        # Q(curr_state, curr_action) = Q(cs, ca) + lr*TD
        # TD = reward + df*Q(next_state, next_action) - Q(cs, ca)
        TD = reward + discount_factor*agent.Q[next_state][next_action] - agent.Q[curr_state][curr_action]
        agent.Q[curr_state][curr_action] =  agent.Q[curr_state][curr_action] + learning_rate*TD

        # update state
        curr_state = next_state
        curr_action = next_action



In [131]:
print(agent.Q)

{0: {'LEFT': 0.9212066922477286, 'RIGHT': 10.005826889511352}, 1: {'LEFT': 0.8400398308320598, 'RIGHT': 12.206447263500301}, 2: {'LEFT': 2.867200788698947, 'RIGHT': 15.767298766757618}, 3: {'LEFT': 0.7321647886244969, 'RIGHT': 18.829251681612565}, 4: {'LEFT': 9.999635646105794, 'RIGHT': 2.71}}


In [132]:
# simulate in the env
env.reset()

while not env.is_done():
    action = agent.sample_action(env.state)
    env.step(action=action)
    print(env.state)

1
2
3
4
